In [3]:
import re

from tqdm import tqdm
from anytree import Node, RenderTree, search
from collections import defaultdict
from textblob import TextBlob

from litecoder.db import City, session

In [4]:
City.query.count()

344249

In [10]:
b = TextBlob('I grew up in Tuscaloosa, AL in the 90s.')

In [15]:
type(list(b.tokens)[0])

textblob.blob.Word

In [8]:
TextBlob.tokenize?

Signature: TextBlob.tokenize(self, tokenizer=None)
Docstring:
Return a list of tokens, using ``tokenizer``.

:param tokenizer: (optional) A tokenizer object. If None, defaults to
    this blob's default tokenizer.
File:      ~/Projects/litecoder/env/lib/python3.6/site-packages/textblob/blob.py
Type:      function
